<a href="https://colab.research.google.com/github/afaq5587/OPENAI-SDK-AGENTS/blob/main/Agent_using_tools_handsoff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq openai-agents
!pip install -Uq litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 37.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai-agents 0.0.14 requires openai>=1.76.0, but you have openai 1.75.0 which is incompatible.


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from google.colab import userdata
from pydantic import BaseModel
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig
)

In [ ]:
gemini_api_key = userdata.get("GEMINI_API_KEY")

from agents import set_default_openai_client, set_tracing_disabled


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)
set_default_openai_client(external_client)
set_tracing_disabled(True)

In [ ]:
import asyncio
import os
import requests
from agents import Agent, Runner, function_tool
from litellm import completion

OPENWEATHERMAP_API_KEY = userdata.get("OPENWEATHERMAP_API_KEY")
TAVILY_API_KEY = userdata.get("TAVILY_API_KEY")

@function_tool
def get_weather(city: str) -> str:
    """Fetch the current weather for a given city using OpenWeatherMap API.

    Args:
        city: The name of the city to fetch the weather for.
    """
    api_key = OPENWEATHERMAP_API_KEY
    if not api_key:
        return "Error: OpenWeatherMap API key not set."

    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        weather = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        return f"The weather in {city} is {weather} with a temperature of {temp}°C."
    except requests.RequestException as e:
        return f"Error fetching weather for {city}: {str(e)}"

# Create the agent with Gemini model
agent = Agent(
    name="WeatherAssistant",
    instructions="You are a helpful assistant that provides weather information for cities. Use the get_weather tool to fetch current weather data when asked about the weather.",
    model=model,
    tools=[get_weather]
)

# Main function to run the agent
async def main():
    result = await Runner.run(agent, "What's the weather in Pakistan(islamabad)?")
    print(result.final_output)

# Run the agent
if __name__ == "__main__":
    asyncio.run(main())

The weather in Islamabad is clear sky with a temperature of 37.13°C.



In [ ]:
import asyncio
import os
import requests
from agents import Agent, Runner, function_tool
from litellm import completion

# Set environment variables for API keys
# Replace with your actual API keys
# os.environ["GEMINI_API_KEY"] = "your-gemini-api-key-here"
# os.environ["OPENWEATHERMAP_API_KEY"] = "your-openweathermap-api-key-here"
# os.environ["TAVILY_API_KEY"] = "your-tavily-api-key-here"

# Define the third-party weather tool
@function_tool
def get_weather(city: str) -> str:
    """Fetch the current weather for a given city using OpenWeatherMap API.

    Args:
        city: The name of the city to fetch the weather for.
    """
    api_key = OPENWEATHERMAP_API_KEY
    if not api_key:
        return "Error: OpenWeatherMap API key not set."

    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        weather = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        return f"The weather in {city} is {weather} with a temperature of {temp}°C."
    except requests.RequestException as e:
        return f"Error fetching weather for {city}: {str(e)}"

# Define the Tavily search tool
@function_tool
def tavily_search(query: str) -> str:
    """Perform a web search using the Tavily API.

    Args:
        query: The search query to send to Tavily.
    """
    api_key = TAVILY_API_KEY
    if not api_key:
        return "Error: Tavily API key not set."

    url = "https://api.tavily.com/search"
    payload = {
        "api_key": api_key,
        "query": query,
        "search_depth": "basic",
        "max_results": 5
    }
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        data = response.json()
        results = data.get("results", [])
        if not results:
            return "No results found for the query."

        # Format the top results
        formatted_results = []
        for idx, result in enumerate(results[:3], 1):
            title = result.get("title", "No title")
            snippet = result.get("content", "No content")
            url = result.get("url", "No URL")
            formatted_results.append(f"{idx}. {title}\n   {snippet}\n   Source: {url}")

        return "\n\n".join(formatted_results)
    except requests.RequestException as e:
        return f"Error performing Tavily search: {str(e)}"

# Create the agent with Gemini model
agent = Agent(
    name="WeatherAndSearchAssistant",
    instructions="You are a helpful assistant that provides weather information for cities and answers general questions using web search. Use the get_weather tool to fetch current weather data when asked about the weather. Use the tavily_search tool to perform a web search for other queries.",
    model=model,  # Adjust model as needed
    tools=[get_weather, tavily_search]
)

# Main function to run the agent
async def main():
    # Example query combining weather and search
    result = await Runner.run(agent, "What's the weather in Islamabad, Pakistan, and what's the latest news there?")
    print(result.final_output)

# Run the agent
if __name__ == "__main__":
    asyncio.run(main())

The weather in Islamabad is clear sky with a temperature of 37.13°C. According to DAWN.COM, a trusted news outlet in Pakistan, some of the latest news includes the Higher Education Commission (HEC) and the World Bank beginning their 12th program, Islamabad's New Blue Area is set to host an event, and Quetta Gladiators defeated Islamabad United in a cricket match.



In [ ]:
import asyncio
import os
import requests
from agents import Agent, Runner, function_tool
from litellm import completion
OPENWEATHERMAP_API_KEY = userdata.get("OPENWEATHERMAP_API_KEY")
TAVILY_API_KEY = userdata.get("TAVILY_API_KEY")



@function_tool
def get_weather(city: str) -> str:
    """Fetch the current weather for a given city using OpenWeatherMap API.

    Args:
        city: The name of the city to fetch the weather for.
    """
    api_key = OPENWEATHERMAP_API_KEY
    if not api_key:
        return "Error: OpenWeatherMap API key not set."

    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        weather = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        return f"The weather in {city} is {weather} with a temperature of {temp}°C."
    except requests.RequestException as e:
        return f"Error fetching weather for {city}: {str(e)}"

@function_tool
def tavily_search(query: str) -> str:
    """Perform a web search using the Tavily API.

    Args:
        query: The search query to send to Tavily.
    """
    api_key = TAVILY_API_KEY
    if not api_key:
        return "Error: Tavily API key not set."

    url = "https://api.tavily.com/search"
    payload = {
        "api_key": api_key,
        "query": query,
        "search_depth": "basic",
        "max_results": 5
    }
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        data = response.json()
        results = data.get("results", [])
        if not results:
            return "No results found for the query."

        formatted_results = []
        for idx, result in enumerate(results[:3], 1):
            title = result.get("title", "No title")
            snippet = result.get("content", "No content")
            url = result.get("url", "No URL")
            formatted_results.append(f"{idx}. {title}\n   {snippet}\n   Source: {url}")

        return "\n\n".join(formatted_results)
    except requests.RequestException as e:
        return f"Error performing Tavily search: {str(e)}"

general_assistant = Agent(
    name="GeneralAssistant",
    instructions="You are a versatile assistant that can provide weather information and perform web searches. Use the get_weather tool for weather-related queries and the tavily_search tool for general knowledge or news queries.",
    model=model,
    tools=[get_weather, tavily_search]
)
tool = general_assistant.as_tool(tool_name="GeneralAssistant",tool_description="You are a versatile assistant that can provide weather information and perform web searches. Use the get_weather tool for weather-related queries and the tavily_search tool for general knowledge or news queries.",)

triage_agent  = Agent(
    name="triage agent",
    instructions="You are a helpful assistant that primarily delegates tasks to the GeneralAssistant. If the query involves weather or general knowledge, hand it off to the GeneralAssistant using the handoff tool. Only handle queries directly if they are about coordinating multiple tasks or summarizing results from the GeneralAssistant.",
    model=model,
    tools=[tool]
)

async def main():
    query = ""
    while query!="exit":
      print("wellcome! if you want to exit from here enter (exit)")
      query = input("Enter your query!.  ")
      result = await Runner.run(triage_agent , query)
      print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

wellcome! if you want to exit from here enter (exit)
Enter your query!.  what is happening  in india
Here are some of the top news stories from India:

*   **Ceasefire After 4 Days:** There are reports about a ceasefire agreement between India and Pakistan.
*   **Operation Sindoor:** India has said that Operation Sindoor is ongoing, which marks a new approach in India’s response to cross-border terrorism.
*   **PM Modi's Visit to Adampur Air Base:** Prime Minister Narendra Modi visited Adampur Air Base in Punjab and asserted that India’s drones and missiles gave sleepless nights to Pakistan.

wellcome! if you want to exit from here enter (exit)
Enter your query!.  exit
Okay, goodbye! Let me know if you need anything else in the future.

